## Imports

In [2]:
import torch
import pandas as pd
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
from torch.utils.data import DataLoader
from torch import nn
import time
from torch.utils.data.dataset import random_split

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


c:\Users\waqasali\Anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Datasets and Pre-process

In [3]:
# Load data from csv file
train_df = pd.read_csv('../data/train_berkeley.csv') 
test_df = pd.read_csv('../data/test_berkeley.csv')

# Add labels to the dataframe
train_df['label'] = 0
for i in range(len(train_df)):
    if train_df.discourse_effectiveness[i] == 'Adequate':
        train_df['label'][i] = 0 
    elif train_df.discourse_effectiveness[i] == 'Effective':    
        train_df['label'][i] = 1
    elif train_df.discourse_effectiveness[i] == 'Ineffective':    
        train_df['label'][i] = 2

test_df['label'] = 0
for i in range(len(test_df)):
    if test_df.discourse_effectiveness[i] == 'Adequate':
        test_df['label'][i] = 0 
    elif test_df.discourse_effectiveness[i] == 'Effective':    
        test_df['label'][i] = 1
    elif test_df.discourse_effectiveness[i] == 'Ineffective':    
        test_df['label'][i] = 2

# take label and text from dataframe and convert into an iterator object
train_iter_list = []
for i in range(len(train_df)):
    train_iter_list.append([train_df.label[i], train_df.discourse_text[i]])
train_iter = (x for x in train_iter_list) 
train_iter = torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe(train_iter)

test_iter_list = []
for i in range(len(test_df)):
    test_iter_list.append([test_df.label[i], test_df.discourse_text[i]])
test_iter = (x for x in test_iter_list) 
test_iter = torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe(test_iter)

## Data Processing Pipeline

In [4]:
# build vocabulary with the raw training dataset
tokenizer = get_tokenizer('basic_english')

counter = Counter()
for (label, line) in train_iter:
    counter.update(tokenizer(line))
vocab = vocab(counter, min_freq = 3, specials=["<unk>"])
vocab.set_default_index(vocab['<unk>'])

# build text and lables pipelines
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x)

## Generate Data Batch and Iterator 

In [5]:

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)    

train_iter_list = []
for i in range(len(train_df)):
    train_iter_list.append([train_df.label[i], train_df.discourse_text[i]])
train_iter = (x for x in train_iter_list) 
train_iter = torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe(train_iter)

dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)


## Build Model

In [6]:

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

## Initiate an Instance

In [7]:
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training
EMSIZE = 64

train_iter_list = []
for i in range(len(train_df)):
    train_iter_list.append([train_df.label[i], train_df.discourse_text[i]])
train_iter = (x for x in train_iter_list) 
train_iter = torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe(train_iter)

num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
model = TextClassificationModel(vocab_size, EMSIZE, num_class).to(device)

## Define Training Loop




In [8]:

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

## Split the Dataset and Run the Model Training

In [9]:
train_iter_list = []
for i in range(len(train_df)):
    train_iter_list.append([train_df.label[i], train_df.discourse_text[i]])
train_iter = (x for x in train_iter_list) 
train_iter = torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe(train_iter)

test_iter_list = []
for i in range(len(test_df)):
    test_iter_list.append([test_df.label[i], test_df.discourse_text[i]])
test_iter = (x for x in test_iter_list) 
test_iter = torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe(test_iter)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_dataset = list(train_iter)
test_dataset = list(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f}'.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

-----------------------------------------------------------
| end of epoch   1 | time:  6.30s | valid accuracy    0.621
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  6.68s | valid accuracy    0.613
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  6.63s | valid accuracy    0.625
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  6.62s | valid accuracy    0.627
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  6.68s | valid accuracy    0.627
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  6.60s | vali

## Model Evaluation on  Test Dataset



In [10]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.612


In [11]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

620035

In [16]:
test_iter_list = []
for i in range(len(test_df)):
    test_iter_list.append([test_df.label[i], test_df.discourse_text[i]])
test_iter = (x for x in test_iter_list) 
test_iter = torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe(test_iter)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)


test_dataset = list(test_iter)

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)



model.eval()
with torch.no_grad():
    for idx, (label, text, offsets) in enumerate(dataloader):
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
print("Loss: ", loss.item())

Loss:  3.3966753482818604
